In [12]:
# Step 1: Import Necessary Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import joblib

# Step 2: Load and Preprocess the Data
# Load data from the Excel file (adjust the path if needed)
file_path = 'crest containers db.xlsx'
excel_file = pd.ExcelFile(file_path)

# Load necessary sheets
order_list = excel_file.parse('OrderList')
freight_rates = excel_file.parse('FreightRates')
wh_costs = excel_file.parse('WhCosts')

# Convert 'Order Date' in the OrderList sheet to datetime format
order_list['Order Date'] = pd.to_datetime(order_list['Order Date'])

# Encode categorical variables
order_list_encoded = pd.get_dummies(order_list, columns=['Origin Port', 'Carrier', 'Service Level', 'Customer', 'Plant Code', 'Destination Port'])

# Feature Engineering: Create 'Shipping Duration'
order_list_encoded['Shipping Duration'] = order_list_encoded['Ship Late Day count'] + order_list_encoded['Ship ahead day count']
order_list_encoded.drop(['Ship Late Day count', 'Ship ahead day count'], axis=1, inplace=True)

# Prepare data for modeling
X = order_list_encoded.drop(['Order ID', 'Order Date', 'Product ID', 'TPT'], axis=1)  # Features
y = order_list_encoded['TPT']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Train the Model
# Initialize the Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Predict on the test data
y_pred = rf_model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Root Mean Squared Error (RMSE): {rmse}")

# Step 4: Save the Trained Model and Feature List
# Save the trained model
joblib.dump(rf_model, 'random_forest_model.pkl')

# Save the list of feature names used during training
feature_list = X.columns.tolist()
joblib.dump(feature_list, 'feature_list.pkl')

print("Model and feature list saved successfully.")


Mean Absolute Error (MAE): 0.028589256646771567
Root Mean Squared Error (RMSE): 0.2785493582491427
Model and feature list saved successfully.


In [2]:
import pandas as pd
import joblib

# Load the original data
file_path = './crest containers db.xlsx'
excel_file = pd.ExcelFile(file_path)
order_list = excel_file.parse('OrderList')

# Convert 'Order Date' to datetime format
order_list['Order Date'] = pd.to_datetime(order_list['Order Date'])

# Load the trained Random Forest model
model = joblib.load('random_forest_model.pkl')

# Encode categorical variables using the same method used during training
order_list_encoded = pd.get_dummies(order_list, columns=['Origin Port', 'Carrier', 'Service Level', 'Customer', 'Plant Code', 'Destination Port'])

# Feature Engineering: Create 'Shipping Duration'
order_list_encoded['Shipping Duration'] = order_list_encoded['Ship Late Day count'] + order_list_encoded['Ship ahead day count']
order_list_encoded.drop(['Ship Late Day count', 'Ship ahead day count'], axis=1, inplace=True)

# Prepare data for prediction (excluding unnecessary columns)
X = order_list_encoded.drop(['Order ID', 'Order Date', 'Product ID', 'TPT'], axis=1)

# Predict delivery time for the entire dataset
order_list_encoded['Predicted Delivery Time'] = model.predict(X)

# Merge predictions with original data for better understanding
results = order_list.join(order_list_encoded[['Predicted Delivery Time']])

# Save the results to a CSV file for Power BI
results.to_csv('logistics_predictions.csv', index=False)
print("Data with predictions saved to 'logistics_predictions.csv'.")


Data with predictions saved to 'logistics_predictions.csv'.


In [1]:
import pandas as pd
import joblib
from datetime import datetime

# Load the original data
file_path = './crest containers db.xlsx'
excel_file = pd.ExcelFile(file_path)
order_list = excel_file.parse('OrderList')

# Convert 'Order Date' to datetime format
order_list['Order Date'] = pd.to_datetime(order_list['Order Date'])

# Load the trained Random Forest model
model = joblib.load('random_forest_model.pkl')

# Encode categorical variables using the same method used during training
order_list_encoded = pd.get_dummies(order_list, columns=['Origin Port', 'Carrier', 'Service Level', 'Customer', 'Plant Code', 'Destination Port'])

# Feature Engineering: Create 'Shipping Duration'
order_list_encoded['Shipping Duration'] = order_list_encoded['Ship Late Day count'] + order_list_encoded['Ship ahead day count']
order_list_encoded.drop(['Ship Late Day count', 'Ship ahead day count'], axis=1, inplace=True)

# Prepare data for prediction (excluding unnecessary columns)
X = order_list_encoded.drop(['Order ID', 'Order Date', 'Product ID', 'TPT'], axis=1)

# Predict delivery time for the entire dataset
order_list_encoded['Predicted Delivery Time'] = model.predict(X)

# Merge predictions with original data for better understanding
results = order_list.join(order_list_encoded[['Predicted Delivery Time']])

# Save the results to a CSV file for Power BI
output_filename = f'logistics_predictions_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv'
results.to_csv(output_filename, index=False)
print(f"Data with predictions saved to '{output_filename}'.")


Data with predictions saved to 'logistics_predictions_20240911_221359.csv'.


https://chatgpt.com/share/0a288a96-ec2b-4123-a9f8-8e5a9f96268a